# Nh5 Event Model

## Notes on building an event model based in hdf5, pytabes and pandas

In [1]:
from __future__ import print_function

### GATE event model

GATE event model (EM) can be exposed to python (via pyROOT). On the other hand:

1. Since the GATE objects were designed in C++, the objects exposed to python are "C++ translations" performed by pyROOT. This comes at the cost of loosing reflection/introspection. pyROOT objects can only be manipulated using their C++ interfaces, which are often not obviously translated to python (maps is a good example). 

2. GATE objects are stored using ROOT. Depending on your faith you may consider this a blessing or a curse. 

3. A description of GATE EVM is here:

http://next.ific.uv.es:8888/nextsw/dstbuilder/wikis/EventModel

## Nh5 Event Model

### Raw Data objects

#### Monte Carlo Raw  Data (MCRD) True Waveforms (TWF), Raw Waveforms (RWF),  and Corrected (restored) Waveforms (CWF) 

1. For detailed studies of the Energy Plane FEE, waveforms (bins of 1 ns) in photoelectrons (pes) are neeed. We refer to these as Monte Carlo Raw Data (MCRD). The MCRD is the input to the Simulation of the Energy Plane Response (SIERPE), which convolutes the MCRD with the response of the Energy plane FEE and outputs True Waveforms and Raw Waveforms.

2. TWF are simply MCRD decimated by the DAQ (e.g, sampled each 25 ns). TWF would be the output of an ideal FEE+DAQ of gain = 1 (e.g, simple decimation). They are still expressed in pes. 

3. RWF are the output of the FEE + DAQ. They are obtained by convoluting the MCRD with the LPF and HPF characteristic of the Energy Plane FEE. They are also decimated by the DAQ. The output of the FEE + DAQ is in adc counts. 

4. CWF are the output of the Digital Baseline Restoration (BLR) algorithm. The output of the BLR is also in adc counts.

5. CWF can be expressed in pes dividing by the adc_to_pes gain factor. The gain can be computed dividing the area of the CWF by the TWF. 



http://localhost:8888/notebooks/Notebooks/SIERPE.ipynb

### Structure of the DST

1. For detailed MC simulation of the EP one needs to produce (using NEXUS) MCRD files. In spite of the fact that 1 ns waveforms are very large (thus MCRD files weight about 10 MB per event if not compressed), compression is very effective here, since most of the WF contents are zeros. The actual size of the event is of the order of 250 kb. 
2. DIOMIRA takes as an imput MCRD files and writes DST files whih contain TWF and RWF, both of them procued by the SIERPE simulation of the EP. In spite of the fact that TWF25ns have 25 times less elements than TWF1ns, the resulting files, at the same level of compression are heavier, because compression is less effective (there are many numbers close to zero but not exactly zero, due to noise). The file weight about 0.7 kb per event. 
3. RWF are passed by DBLR which produces corrected (baseline restored) Waveforms. The resulting CWF are also stored in the DST file. 
4. Notice that NEXUS can produce directly CWF, corresponding to the case where DBLR is performed directly in the FPGA (in this case the DAQ also produces CWF). 

### Decimate or resum (rebin)

Nexus + art produce WF in bins of 25 ns, while RWF, CWF and TWF are DECIMATED in bins of 25 ns from the original MCRD WF. The difference is relevant, in the sense that the DAQ produces also decimated WF. In practice is very simple to convert a decimated WF into a "rebinned" WF, simply: rebinned = decimated * 25. It is imoportant to carry the bookeeping right. The total energy of a decimated WF is sum(wF)x 25 (25 ns is the decimation factor), while the total energy of a rebinned WF is just the sum(wF). 